In [8]:
#!pip3 install dwave-system

In [9]:
import pandas as pd 
import numpy as np
from scipy.special import comb
import math
from neal import SimulatedAnnealingSampler
import itertools
import random
import matplotlib.pyplot as plt
from dwave.system.samplers import DWaveCliqueSampler
from dwave.system.composites import EmbeddingComposite
from dwave.system import ReverseBatchStatesComposite, ReverseAdvanceComposite
from dwave.embedding.chain_strength import uniform_torque_compensation
import dimod
import timeit

In [10]:
def calc_marginals(df):                   
    return np.array([                      
        sum(df['Y']),                     
        np.dot(df['Y'], df['LI']),      
        np.dot(df['Y'], df['SEX']),      
        np.dot(df['Y'], df['AOP']),      
    ])     

In [11]:
def make_Hamiltonian(df, t1):
    t_list = calc_marginals(df)
    N=len(df)
    dup_list = [(i, i) for i in range(N)]
    comb_list = [(i, j) for i in range(N) for j in range(i+1, N)]
    
    lin_Y = [1-2*t_list[0] for (i, _) in dup_list] #同じy同士
    quad_Y = [2 for (i, j) in comb_list] #異なるy同士
    num_Y = t_list[0]**2 #数字の二乗
    
    LI = df['LI'].iloc
    lin_LI = [(LI[i] - 2 * t1) * LI[i] for (i, _) in dup_list]
    quad_LI = [2*LI[i] * LI[j] for (i, j) in comb_list]
    num_LI = t1**2
    
    SEX = df['SEX'].iloc
    lin_SEX  = [(SEX[i] - 2 * t_list[2]) * SEX[i] for (i, _) in dup_list]
    quad_SEX  = [2*SEX[i] * SEX[j] for (i, j) in comb_list]
    num_SEX  = t_list[2]**2
    
    AOP = df['AOP'].iloc
    lin_AOP = [(AOP[i] - 2 * t_list[3]) * AOP[i] for (i, _) in dup_list]
    quad_AOP = [2*AOP[i] * AOP[j] for (i, j) in comb_list]
    num_AOP = t_list[3]**2
    
    #lin
    lin_list = [sum(lin) for lin in zip(lin_Y, lin_LI, lin_SEX, lin_AOP)]
    lin = {i: lin_list[i] for (i, _) in dup_list}
    
    #quad
    quad_values = [sum(quad) for quad in zip(quad_Y, quad_LI, quad_SEX, quad_AOP)]
    quad = {ij: quad_values[n] for (n, ij) in enumerate(comb_list)}
    
    #num
    num = num_Y + num_LI + num_SEX + num_AOP
    
    return dimod.BinaryQuadraticModel(lin, quad, num, dimod.Vartype.BINARY)#dic, dic, num

In [12]:
def QA_ReverseAdvanceComposite(df, numreads):
    valid_y_list = {}                                                                   
    valid_y_num = {}
    for t1 in range(0, sum(df['LI'])):
        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        
        bqm = make_Hamiltonian(df, t1)

        dw_sampler = DWaveCliqueSampler(
            endpoint="https://cloud.dwavesys.com/sapi",
            solver = 'DW_2000Q_6',
            token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
        )

        sampler_reverse = ReverseAdvanceComposite(dw_sampler)
        init_samples = {n: y for n, y in enumerate(df['Y'].values.tolist())}

        res = sampler_reverse.sample(bqm,
                                       anneal_schedules=[[[0.0, 1.0], [t, 0.5], [20, 1.0]] for t in (5, 10, 6)],
                                       initial_state=init_samples,
                                       num_reads=numreads,
                                       reinitialize_state=True)

        for canditate_y_info in list(res.record):
            if canditate_y_info[1]==0.:
                if all(list(canditate_y_info[0]) != p for p in valid_y_list[t1]): 
                    valid_y_list[t1].append(list(canditate_y_info[0]))
                    valid_y_num[t1] += 1
                
    return valid_y_list, valid_y_num

In [13]:
def QA_ReverseBatchStatesComposite(df, numreads):
    valid_y_list = {}                                                                   
    valid_y_num = {}
    for t1 in range(0, sum(df['LI'])):
        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        
        bqm = make_Hamiltonian(df, t1)

        dw_sampler = DWaveCliqueSampler(
            endpoint="https://cloud.dwavesys.com/sapi",
            solver = 'DW_2000Q_6',
            token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
        )

        sampler_reverse = ReverseBatchStatesComposite(dw_sampler)
        init_samples = {n: y for n, y in enumerate(df['Y'].values.tolist())}

        res = sampler_reverse.sample(bqm,
                                     initial_state=init_samples,
                                     anneal_schedule=[[0.0, 1.0], [10.0, 0.5], [20, 1.0]],
                                     num_reads=numreads,
                                     reinitialize_state=True)

        for canditate_y_info in list(res.record):
            if canditate_y_info[1]==0.:
                if all(list(canditate_y_info[0]) != p for p in valid_y_list[t1]): 
                    valid_y_list[t1].append(list(canditate_y_info[0]))
                    valid_y_num[t1] += 1
                
    return valid_y_list, valid_y_num

In [14]:
def QA_ReverseBatchStatesComposite_time(df, numreads, time_time):
    valid_y_list = {}                                                                   
    valid_y_num = {}
    for t1 in range(0, sum(df['LI'])):
        valid_y_list[t1] = []
        valid_y_num[t1] = 0
        
        bqm = make_Hamiltonian(df, t1)

        dw_sampler = DWaveCliqueSampler(
            endpoint="https://cloud.dwavesys.com/sapi",
            solver = 'DW_2000Q_6',
            token = "TOKY-1319d5c52b9aa35f34b40feba0cea58a4f5d3c09"
        )

        sampler_reverse = ReverseBatchStatesComposite(dw_sampler)
        init_samples = {n: y for n, y in enumerate(df['Y'].values.tolist())}

        res = sampler_reverse.sample(bqm,
                                     initial_state=init_samples,
                                     anneal_schedule=[[0.0, 1.0], [time_time, 0.5], [20, 1.0]],
                                     num_reads=numreads,
                                     reinitialize_state=True)

        for canditate_y_info in list(res.record):
            if canditate_y_info[1]==0.:
                if all(list(canditate_y_info[0]) != p for p in valid_y_list[t1]): 
                    valid_y_list[t1].append(list(canditate_y_info[0]))
                    valid_y_num[t1] += 1
                
    return valid_y_list, valid_y_num

In [15]:
df6 = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
print(df6)

   Y  LI  SEX  AOP
0  1   1    0    0
1  0   1    1    1
2  0   1    1    0
3  0   1    1    1
4  0   1    0    1
5  1   1    0    1


　## 日記
 - reinitialize_stateを変化させても結果に違いなし
 
## ヒントとなる？
- num_spin_reversal_transforms：アナログエラーを回避できる？スピン反転変換の数を指定。
- auto_scale：ℎ と 𝐽 の値の全範囲を明示的に扱う必要をなくすにはTureにする。
- annealing_time:トンネルエネルギー（または横方向のエネルギー）A(s)を減らし、ハミルトニアンエネルギーB(s)を増やす、そんな時間。sは0~1。
  - トンネルエネルギー: ⊿qと同等。セクション”結合したrf-SQUIDE 量子ビット”で定義されている。
  - ハミルトニアンエネルギー：磁束量子ビット体のペア間で達成可能な最大の相互インダクタンスに関するエネルギー
- reversal distatnce
- max_anneal_schedule_points: PWL(ピースワイズリニア)波形。グローバルアニーリングスケジュールの変更。　pauseやquench(急冷)を導入することで前方向のスケジュールを変更することができる。D-Wave 2000Q システムの場合、スケジュールの最大のポイント数はシステムに依存するらしい。

- 永久電流の二次関数的増加、すなわち、𝐵(𝑡)の二次関数的増加を”annealing_time”パラメータによりスケールさせることができる。
- D-Wave2000Q では、新しく”anneal_schedule”パラメータによって、アニーリングプロセスの途中でポーズ（一時停止）やクエンチ（急冷）が可能。
- ”annealing_time”と”anneal_schedule”パラメータは同時に使えない


- スケジュールの変更は、𝑛 対のポイントからなる PWL 波形で制御される。
- PWL 波形の各セグメントの最大の傾きは、パラメータ Annealing Slope Rangeで指定された上限値

- スピン反転変換はリバース（反転）アニーリングと両立しない

- h leakage, NNN(next-nearest neighbor) coupling

- 埋め込み：チェーンが必要になることも。3変数(量子)をキメラグラフ（4）に埋め込む場合など。
- マイナー埋め込み：論理量子ビットを物理量子ビットにマッピングするプロセス

In [16]:
QA_ReverseAdvanceComposite(df6, 100)

({0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1]], 3: [], 4: [], 5: []},
 {0: 0, 1: 0, 2: 1, 3: 0, 4: 0, 5: 0})

In [71]:
QA_ReverseBatchStatesComposite(df6, 100)

RuntimeError: can't start new thread

In [54]:
for time_time in range(1, 20):
    valid_y_list, valid_y_num = QA_ReverseBatchStatesComposite_time(df6, 20, time_time)
    print(valid_y_num)

RuntimeError: can't start new thread

In [55]:
df16 = pd.read_csv('../../input/ost16.csv', sep=',', index_col=0)
print(df16)

    Y  LI  SEX  AOP
0   1   1    1    0
1   1   1    0    1
2   1   1    1    1
3   1   1    1    1
4   0   1    1    1
5   0   1    1    1
6   1   1    1    0
7   0   1    0    1
8   0   1    1    1
9   0   1    0    1
10  1   0    0    1
11  0   1    1    1
12  0   1    1    1
13  1   0    0    0
14  0   1    1    0
15  1   1    0    0


In [61]:
QA_ReverseAdvanceComposite(df16, 10)

RuntimeError: can't start new thread

In [63]:
QA_ReverseBatchStatesComposite(df16, 10)

RuntimeError: can't start new thread

In [72]:
def time_measurement(df):
    sum_time = 0
    annealing_time = 20
    for t1 in range(0, sum(df['LI'])+1):
        timeit_repeat = timeit.repeat("make_Hamiltonian(df, t1)", number=1, repeat=1, globals={"make_Hamiltonian": make_Hamiltonian, "df": df, "t1": t1})
        sum_time += timeit_repeat[0] + annealing_time
    return sum_time

In [73]:
#==========
#テストコード
#==========
def test_find_valid_y():
    df = pd.read_csv('../../input/ost6.csv', sep=',', index_col=0)
    true_t1 = sum(df['Y'] * df['LI'])
    valid_y_list, valid_y_num = find_valid_y(df,  num_reads=20)
    print(valid_y_list, valid_y_num)
    assert valid_y_num[true_t1] > 0
    
test_find_valid_y()

{0: [], 1: [], 2: [[1, 0, 0, 0, 0, 1], [1, 0, 0, 0, 1, 0]], 3: [], 4: [], 5: [], 6: []} {0: 0, 1: 0, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0}


In [64]:
def test_validity():
    df1 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    df2 = pd.read_csv('../../input/ost6.csv', sep=',',index_col=0)
    new_y = np.array([1, 0, 0, 0, 0, 1])
    df2['Y'] = new_y
    t_list1 = calc_marginals(df1)
    t_list2 = calc_marginals(df2)
    print(t_list1)
    print(t_list2)
    assert np.all(t_list1[[0,2,3]] == t_list2[[0,2,3]]) 

#test_validity()

[2 2 0 1]
[2 2 0 1]
